### Data Science capstone

The objective of this project is being able to predict the severity of an accident.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df=pd.read_csv('https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv')

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Checking the dataset

In [2]:
df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


### We check the columns searching for the correct features to feed the model.

In [3]:
df.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

### We choose a set of features to start checking their impact on the purpose of the project.

In [ ]:
new_df=df[['SEVERITYCODE','ADDRTYPE','COLLISIONTYPE','PERSONCOUNT','INCDTTM','UNDERINFL','INATTENTIONIND','WEATHER','ROADCOND','LIGHTCOND','SPEEDING']]
new_df.head()

### A bit of data manipulation must be done to set the information numerical values for the model.

Starting with the categorical features

In [104]:
from sklearn.preprocessing import LabelEncoder

labelenc = LabelEncoder()

x=new_df[['ADDRTYPE','COLLISIONTYPE','PERSONCOUNT','INCDTTM','UNDERINFL','INATTENTIONIND','WEATHER','ROADCOND','LIGHTCOND','SPEEDING']]

cat_labels=['ADDRTYPE','COLLISIONTYPE','WEATHER','ROADCOND','LIGHTCOND']
for i in cat_labels:
    #print(x[i].value_counts())
    x[i].replace(np.nan, 'Unknown', inplace=True)
    x[i]=labelenc.fit_transform(x[i])
    #print(x[i].value_counts())
    #print('')

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

Following with the more numerical like features

In [ ]:
num_labels=['UNDERINFL','INATTENTIONIND','SPEEDING']
for j in num_labels:
    x[j].replace(np.nan, '0', inplace=True)

x['SPEEDING'].replace('Y', '1', inplace=True)
x['INATTENTIONIND'].replace('Y', '1', inplace=True)
x['UNDERINFL'].replace(['N','Y'], ['1','2'], inplace=True)
x['INCDTTM'] = pd.to_datetime(x['INCDTTM'], format = '%Y-%m-%dT%H:%M:%SZ', errors = 'coerce')

### Preparing the data format

In [106]:
x.drop(['INCDTTM'], axis=1, inplace=True)
x=x.values
y=new_df['SEVERITYCODE'].values

print(x[0:5])
print('')
print(y[0:5])


AttributeError: 'numpy.ndarray' object has no attribute 'drop'

### Splitting of the dataset for the model.

In [107]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=1)
print ('Train set:', x_train.shape,  y_train.shape)
print ('Test set:', x_test.shape,  y_test.shape)

Train set: (155738, 9) (155738,)
Test set: (38935, 9) (38935,)


### Determination of the best suited "k" value for the model.

In [111]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

k_max=40
accuracy = []
for n in range(k_max):
    n+=1 
    model = KNeighborsClassifier(n_neighbors = n).fit(x_train,y_train)
    y_pred=model.predict(x_test)
    accuracy.append(metrics.accuracy_score(y_test, y_pred))

best_k=np.argmax(accuracy)+1
print('k accuracy vector: ',accuracy)
print('The best k value is: ',best_k)

k accuracy vector:  [0.6694233979709773, 0.7182997303197637, 0.7072299987158084, 0.7370746115320406, 0.7249004751508925, 0.7400282522152305, 0.7334275073841017, 0.7404648773597021, 0.7331706690638243, 0.7383588031334275, 0.7353281109541544, 0.7429818928984204, 0.7402337228714524, 0.7447797611403622, 0.7427507384101708, 0.7475536149993579, 0.7440606138435855, 0.746269423397971, 0.7436753563631694, 0.7436496725311417, 0.7441890330037242, 0.7464235263901374, 0.7445229228200848, 0.7435726210350584, 0.7436753563631694, 0.744702709644279, 0.7451136509567228, 0.7480159239758571, 0.7473738281751637, 0.7479131886477463, 0.7475536149993579, 0.7481700269680236, 0.7493514832412996, 0.7477334018235521, 0.7488378066007448, 0.7484782329523565, 0.7477590856555798, 0.7470913060228586, 0.7471940413509696, 0.7475536149993579]
The best k value is:  33


### We set up the model.

In [112]:
def_model = KNeighborsClassifier(n_neighbors = best_k).fit(x_train,y_train)
#neigh
predictions=def_model.predict(x_test)
accuracy.append(metrics.accuracy_score(y_test, predictions))

### Checking the accuracy through a couple of tests

In [113]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score

js_knn=jaccard_similarity_score(y_test, predictions)
f1_knn=f1_score(y_test, predictions)

print('The Jaccard index is: ',js_knn)
print('')
print('The F1-score is:',f1_knn)

The Jaccard index is:  0.7493514832412996

The F1-score is: 0.8385100362396783
